In [5]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [6]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,Vigencia__c, Corredor__c,Pa_s_del_Paciente__c from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "Vigencia__c",
        "Corredor__c",
        "Pa_s_del_Paciente__c",
    ]
]
Empresas_SF.shape


(6848, 10)

In [7]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape


(6834, 5)

In [8]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas.shape

(6854, 15)

In [9]:
Programas_pacientes = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
        "Vigencia__c": "Vigente",
        "Corredor__c": "Corredor",
        "Pa_s_del_Paciente__c": "Pais",
    }
)
Programas_pacientes = Programas_pacientes[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
    ]
]


In [10]:
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

In [ ]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF.shape

In [ ]:
# Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_pacientes,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

In [ ]:
empresas_unidas = pd.merge(
    left=empresas_unidas,
    right=Pacientes_SF,
    how="left",
    left_on="Paciente__c",
    right_on="Id Account Paciente",
)
empresas_unidas.shape


In [ ]:
empresas_unidas = empresas_unidas.rename(columns={"Id": "Id Pacientes Programas"})
empresas_unidas = empresas_unidas[
    [
        "Id Pacientes Programas",
        "Paciente__c",
        "Programas_por_Empresas__c",
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Id Account Paciente",
        "Rut",
        "Nombre",
        "Ocupacion",
        "Cargo",
        "Departamento",
        "Sexo",
        "Telefono",
        "Tipo de Documento",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
    ]
]

empresas_unidas.shape


In [ ]:
# cantidad_de_PAcientes = pd.DataFrame()
def cantidad_gente(corredor):
    empresas_unidas2 = empresas_unidas[empresas_unidas["Corredor"] == corredor]
    empresas_unidas2 = empresas_unidas2.drop_duplicates(
        subset=["Rut", "Empleador", "Poliza", "Nombre Empresa"], keep="last"
    )
    cantidad_de_PAcientes = empresas_unidas2.groupby(["Poliza", "Empleador"])[
        "Paciente__c"
    ].count()
    cantidad_de_PAcientes = cantidad_de_PAcientes.reset_index()
    # cantidad_de_PAcientes = cantidad_de_PAcientes[cantidad_de_PAcientes["Poliza","Empleador","count"]]
    Archivo_gente = "F:/Reportes SF/MetLife/usuarios/" + corredor + "_gente.xlsx"
    cantidad_de_PAcientes.to_excel(Archivo_gente, index=False)
    cantidad_de_PAcientes.shape

In [ ]:
# Citas de Servicio
results = sf.query_all(
    """
    SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,CreatedById,Tipo_de_la_cita__c from ServiceAppointment
    """
)


Citas_de_servicio = pd.DataFrame(results["records"])
Citas_de_servicio["Empresas"] = np.nan
# Citas_de_servicio['RUT__c'].map(pacientessf_empresas.set_index('IdentificationId__pc_x',verify_integrity=False)['Name'])
Citas_de_servicio["Empresas Id"] = np.nan
Citas_de_servicio = Citas_de_servicio[
    [
        "AppointmentNumber",
        "RUT__c",
        "Empresas",
        "CreatedDate",
        "Tipo_de_la_cita__c",
        "Empresas Id",
        "AccountId",
        "Clinica__c",
        "CreatedById",
    ]
]
Citas_de_servicio["AppointmentNumber"] = "F" + Citas_de_servicio["AppointmentNumber"]

Citas_de_servicio.head()

In [ ]:
# Citas de Servicio Orientacion
results = sf.query_all(
    """
    SELECT Id, AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,Tipo_de_la_cita__c,AppointmentNumber,CreatedById from ServiceAppointment
    """
)
Citas_de_servicio2 = pd.DataFrame(results["records"])
Citas_de_servicio2["Clinica__c"] = "Orientación Médica"
Citas_de_servicio2["Empresas"] = np.nan
Citas_de_servicio2["Empresas Id"] = np.nan
Citas_de_servicio2 = Citas_de_servicio2[
    [
        "AppointmentNumber",
        "RUT__c",
        "Empresas",
        "CreatedDate",
        "Tipo_de_la_cita__c",
        "Empresas Id",
        "AccountId",
        "Clinica__c",
        "CreatedById",
    ]
]
Citas_de_servicio2.shape

In [ ]:
# unir dos citas
Gran_Citas_deservicio = pd.concat([Citas_de_servicio, Citas_de_servicio2], axis=0)
Gran_Citas_deservicio = Gran_Citas_deservicio.rename(
    columns={
        "AppointmentNumber": "Codigo Ficha",
        "RUT__c": "Rut",
        "Empresas": "Empleador",
        "CreatedDate": "Fecha",
        "Tipo_de_la_cita__c": "Canal",
        "Empresas Id": "Cod. Empresa",
        "AccountId": "Id Paciente",
        "Clinica__c": "Clínica",
        "CreatedById": "Creado Por",
    }
)

Gran_Citas_deservicio.shape

In [ ]:
# Clinica_deporte
results = sf.query_all(
    """
   SELECT id,Name,Rut__c,Nombre_Empresa__c,CreatedDate,Canal__c, Empresa__c, Account__c,CreatedById from Ficha_Cl_nica_Deporte__c
     """
)
ficha_deporte = pd.DataFrame(results["records"])

ficha_deporte = ficha_deporte[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "CreatedById",
    ]
]
ficha_deporte["clinica"] = "Deporte"
ficha_deporte = ficha_deporte.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "CreatedById": "Creado Por",
    }
)
ficha_deporte.shape

In [ ]:
# Clinica Nutricion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,CreatedById from Ficha_Clinica_Nutrici_n__c 
     """
)
ficha_nutricion = pd.DataFrame(results["records"])

ficha_nutricion = ficha_nutricion[
    [
        "Name",
        "RUT__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "CreatedById",
    ]
]
ficha_nutricion["clinica"] = "Nutrición"

ficha_nutricion = ficha_nutricion.rename(
    columns={
        "Name": "Codigo Ficha",
        "RUT__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "CreatedById": "Creado Por",
    }
)


ficha_nutricion.shape

In [ ]:
# Clinica AppSex
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId,CreatedById from Ficha_Cl_nica_ApsSex__c 
     """
)
ficha_Apssex = pd.DataFrame(results["records"])

ficha_Apssex = ficha_Apssex[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "CreatedById",
    ]
]
ficha_Apssex["clinica"] = "Asesoría Psicologíca en Salud Sexual"
ficha_Apssex = ficha_Apssex.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "CreatedById": "Creado Por",
    }
)

ficha_Apssex.shape

In [ ]:
# Clinica Psicologia
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId,Motivo_de_consulta_manifiesto__c,CreatedById from Ficha_Cl_nica_Asistencia_Emocional__c 
     """
)
ficha_Psicologia = pd.DataFrame(results["records"])

ficha_Psicologia = ficha_Psicologia[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "Motivo_de_consulta_manifiesto__c",
        "CreatedById",
    ]
]
ficha_Psicologia["clinica"] = "Psicología"
ficha_Psicologia.Motivo_de_consulta_manifiesto__c.fillna(0, inplace=True)
ficha_Psicologia["Motivo_de_consulta_manifiesto__c"] = np.where(
    ficha_Psicologia["Motivo_de_consulta_manifiesto__c"] == 0,
    "Otras Derivaciones",
    ficha_Psicologia["Motivo_de_consulta_manifiesto__c"],
)
# ficha_Psicologia['77'] = len(ficha_Psicologia['Motivo_de_consulta_manifiesto__c'])
ficha_Psicologia = ficha_Psicologia.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "CreatedById": "Creado Por",
        "Motivo_de_consulta_manifiesto__c": "Motivo de consulta",
    }
)

ficha_Psicologia.shape

In [ ]:
# Clinica Sueño
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId,CreatedById from Ficha_Cl_nica_del_Sue_o__c 
     """
)
ficha_Sueno = pd.DataFrame(results["records"])
ficha_Sueno = ficha_Sueno[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "CreatedById",
    ]
]
ficha_Sueno["clinica"] = "Clínica Del Sueño"
ficha_Sueno = ficha_Sueno.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "CreatedById": "Creado Por",
    }
)
ficha_Sueno.shape

In [ ]:
# Clinica PAciente Cronico
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,CreatedById from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c  
     """
)
ficha_Cronico = pd.DataFrame(results["records"])
ficha_Cronico = ficha_Cronico[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "CreatedById",
    ]
]
ficha_Cronico["clinica"] = "Pacientes Crónicos"
ficha_Cronico = ficha_Cronico.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "CreatedById": "Creado Por",
    }
)
ficha_Cronico.shape

In [ ]:
# Clinica Telemedicina
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,CreatedById from Ficha_Cl_nica_Telemedicina__c   
     """
)
ficha_Telemedicina = pd.DataFrame(results["records"])
ficha_Telemedicina = ficha_Telemedicina[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "CreatedById",
    ]
]
ficha_Telemedicina["clinica"] = "Telemedicina"
ficha_Telemedicina = ficha_Telemedicina.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "CreatedById": "Creado Por",
    }
)
ficha_Telemedicina.shape

In [ ]:
# Clinica Veterinaria
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,CreatedById from Ficha_Cl_nica_Veterinaria__c    
     """
)
ficha_Veterinaria = pd.DataFrame(results["records"])
ficha_Veterinaria = ficha_Veterinaria[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "CreatedById",
    ]
]
ficha_Veterinaria["clinica"] = "Veterinaria"
ficha_Veterinaria = ficha_Veterinaria.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "CreatedById": "Creado Por",
    }
)
ficha_Veterinaria.shape

In [ ]:
# Clinica Orientacion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,CreatedById from Ficha_Orientaci_n_Medica__c     
     """
)
ficha_Orientacion = pd.DataFrame(results["records"])
ficha_Orientacion = ficha_Orientacion[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "CreatedById",
    ]
]
ficha_Orientacion["clinica"] = "Orientación Médica"
ficha_Orientacion = ficha_Orientacion.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "CreatedById": "Creado Por",
    }
)
ficha_Orientacion.shape

In [ ]:
# Clinica Front
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,CreatedById from Ficha_Front__c     
     """
)
ficha_Front = pd.DataFrame(results["records"])

ficha_Front = ficha_Front[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "CreatedById",
    ]
]
ficha_Front["clinica"] = "Orientación Médica"
ficha_Front = ficha_Front.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "CreatedById": "Creado Por",
    }
)
ficha_Front.shape

In [ ]:
# Clinica_general
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c,Account__c,Programa__c,CreatedById   from Ficha_Clinica_General__c 
     """
)
ficha_general = pd.DataFrame(results["records"])

ficha_general = ficha_general[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "Programa__c",
        "CreatedById",
    ]
]

ficha_general = ficha_general.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "Programa__c": "Clínica",
        "CreatedById": "Creado Por",
    }
)
ficha_general.shape

In [ ]:
# Clinica Ingreso deportivo
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,CreatedById from Ficha_Cl_nica_Ingreso_Deportivo__c    
     """
)
ficha_Ingreso_dep = pd.DataFrame(results["records"])

ficha_Ingreso_dep = ficha_Ingreso_dep[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "CreatedById",
    ]
]
ficha_Ingreso_dep["clinica"] = "Ingreso Deporte Seguro"
ficha_Ingreso_dep = ficha_Ingreso_dep.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "CreatedById": "Creado Por",
    }
)
ficha_Ingreso_dep.shape

In [ ]:
Clinicas = pd.concat(
    [
        ficha_Front,
        ficha_Ingreso_dep,
        ficha_Orientacion,
        ficha_Veterinaria,
        ficha_Telemedicina,
        ficha_Cronico,
        ficha_Sueno,
        ficha_Psicologia,
        ficha_Apssex,
        ficha_nutricion,
        ficha_deporte,
        ficha_general,
    ],
    axis=0,
)
Clinicas.shape

In [ ]:
Reortes_general = pd.concat([Clinicas, Gran_Citas_deservicio], axis=0)
Reortes_general.shape